In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

Using polar coordinates centered on the lens galaxy, the combined lens potential can be written as

$$\phi (r, \theta) = b r f(\theta) + \frac{r^2}{2} (\gamma_c \cos{2 \theta} + \gamma_s \sin{2 \theta} )$$

where

$$f(\theta) = \left[ 1 - \epsilon \cos{2 ( \theta - \theta_0)} \right]^{1/2}$$

The deflection vector $\nabla \phi$ has cartesian components 

\begin{gather}
\nabla_x \phi = \frac{b}{f(\theta)} [\cos \theta - \epsilon \cos{(\theta - 2 \theta_0)}] + \gamma_c r \cos \theta + \gamma_s r \sin \theta \\
\nabla_y \phi = \frac{b}{f(\theta)} [\sin \theta - \epsilon \sin{(\theta - 2 \theta_0)}] + \gamma_s r \cos \theta - \gamma_c r \sin \theta
\end{gather}

The gravitational lens equation has the form

$$\vec{u} = \vec{x} - \nabla \phi (\vec{x})$$

which is really a set of two equations.
\begin{gather}
u = x - \nabla_x \phi (\vec{x}) \\
v = y - \nabla_y \phi (\vec{x})
\end{gather}
We need a penalty function $\chi^2$ that will determine the parameters $b, \epsilon, \gamma_c, \gamma_s, \theta_0$ in the lens potential $\phi$ and the parameters $u,v$, the position of the source. Let $\vec{x}_i, \sigma_i, \, i=0,1,2,3$ be the positions and uncertainties of the four images. Based on Keeton (2010, Gen.Rel.Grav., 42, 2151) we will define our $\chi^2$ function to be in the source plane. This eliminates the need for solving the lens equation (which is computationally expensive), and is a fine apprixmation given how small our uncertainties are. Let $\vec{\mu}_i = (\mu_i, \nu_i)$ be the position of the source as calculated from the lens equation using the $i$th image position. Then we can define our penalty function to be
$$ \chi^2 = \sum_{i=0}^3 \frac{1}{\sigma_i^2} \left( \vec{u} - \vec{\mu}_i \right)^2$$

In [7]:
def poltocart(r, theta):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return (x, y)

def carttopol(x, y):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan(y, x)
    return (r, theta)


def f(theta, eps, theta0): # defining f(θ)
    return (1 - eps * np.cos(2 * (theta - theta0)))**(1/2)


def dxphi(rtheta, b, eps, gc, gs, theta0): # deflection vector x component
    r, theta = rtheta
    return b / f(theta, eps, theta0) * (np.cos(theta) - eps * np.cos( theta - 2 * theta0)) \
        + gc * r * np.cos(theta) + gs * r * np.sin(theta)


def dyphi(rtheta, b , eps, gc, gs, theta0): #deflection vector y component
    r, theta = rtheta
    return b / f(theta, eps, theta0) * (np.sin(theta) - eps * np.sin( theta - 2 * theta0)) \
        + gs * r * np.cos(theta) - gc * r * np.sin(theta)


def calcsource(params, rtheta): # calculates the source given the ϕ-params and an image position
    b, eps, gc, gs, theta0, _, _ = params
    r, theta = rtheta
    x, y = poltocart(r, theta)
    mu = x - dxphi(rtheta, b, eps, gc, gs, theta0)
    nu = y - dyphi(rtheta, b, eps, gc, gs, theta0)
    return (mu, nu)

    
def lnprob(params, rthetasigma): # our χ^2 function
    b, eps, gc, gs, theta0, u, v = params
    r, theta, sigma = rthetasigma
    chi2 = 0
    for i in range(3):
        mu, nu = calcsource(params, rtheta[i])
        chi2 += ((u - mu)**2 + (v - nu)**2) / sigma[i]**2
    return chi2